# 🧹 Nettoyage des Données — Morocco Student Data Pool

Ce notebook effectue le nettoyage du dataset **Morocco_Student_Data_Pool.csv**.

### Pipeline du projet :
1. **`analyze_data.ipynb`** → Aperçu initial des données brutes
2. **`clean_data.ipynb`** ← (ce notebook) Nettoyage et préparation
3. **`EDA_Advancedا.ipynb`** → Analyse exploratoire avancée

### Étapes de nettoyage :
1. Chargement et aperçu initial
2. Correction du décalage de colonnes
3. Suppression des colonnes entièrement vides
4. Imputation des valeurs manquantes (moyenne / médiane / mode)
5. Conversion des dates
6. Normalisation des colonnes textuelles (hors dates et IDs)
7. Validation des colonnes numériques (notes, taux)
8. Détection et traitement des outliers
9. Résumé et export

> **⚠️ Principe clé** : Les colonnes avec des valeurs manquantes sont imputées par la **moyenne** (distribution symétrique) ou la **médiane** (distribution asymétrique), et non supprimées, car chaque variable peut contenir de l'information utile pour la prédiction.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées")

✅ Bibliothèques importées


## 1. Chargement des données

In [2]:
df = pd.read_csv('Morocco_Student_Data_Pool.csv', low_memory=False)

shape_avant = df.shape
missing_avant = df.isnull().sum().sum()

print(f"📊 Dataset chargé")
print(f"   Lignes : {df.shape[0]:,}  |  Colonnes : {df.shape[1]}")
print(f"   Valeurs manquantes : {missing_avant:,} ({missing_avant / (df.shape[0] * df.shape[1]) * 100:.2f}%)")

📊 Dataset chargé
   Lignes : 10,000  |  Colonnes : 286
   Valeurs manquantes : 204,699 (7.16%)


## 2. Correction du décalage de colonnes

Le CSV brut contient un **décalage d'une colonne** vers la gauche dans les dernières colonnes (après `performance_cible`).  
Par exemple, `date_collecte` contient en réalité les codes collecteurs (`COL008`), et les vraies dates se trouvent dans `intervention_necessaire`.

On corrige ce décalage en réalignant les données vers leurs bonnes colonnes.

In [3]:
shifted_cols = [
    'probabilite_reussite',    # contient → niveau_risque
    'niveau_risque',           # contient → intervention_necessaire
    'intervention_necessaire', # contient → date_collecte
    'date_collecte',           # contient → id_collecteur
    'id_collecteur',           # contient → statut_verification
    'statut_verification',     # contient → date_mise_a_jour
]

cols_exist = all(c in df.columns for c in shifted_cols)
if cols_exist and str(df['date_collecte'].iloc[0]).startswith('COL'):
    print("⚠️ Décalage de colonnes détecté ! Correction en cours...")
    
    saved = {col: df[col].copy() for col in shifted_cols}
    
    df['niveau_risque'] = saved['probabilite_reussite']
    df['intervention_necessaire'] = saved['niveau_risque']
    df['date_collecte'] = saved['intervention_necessaire']
    df['id_collecteur'] = saved['date_collecte']
    df['statut_verification'] = saved['id_collecteur']
    df['probabilite_reussite'] = np.nan  # données d'origine perdues
    
    print("✅ Colonnes réalignées :")
    print(f"   date_collecte[0] = {df['date_collecte'].iloc[0]}")
    print(f"   id_collecteur[0] = {df['id_collecteur'].iloc[0]}")
    print(f"   intervention_necessaire[0] = {df['intervention_necessaire'].iloc[0]}")
else:
    print("✅ Pas de décalage détecté — colonnes correctes.")

⚠️ Décalage de colonnes détecté ! Correction en cours...
✅ Colonnes réalignées :
   date_collecte[0] = 2026-01-15
   id_collecteur[0] = COL008
   intervention_necessaire[0] = Oui


## 3. Suppression des colonnes entièrement vides (100% NaN)

In [4]:
missing_percent = df.isnull().mean()
cols_100_missing = missing_percent[missing_percent == 1.0].index.tolist()

print(f"🗑️ {len(cols_100_missing)} colonnes avec 100% de NaN supprimées :")
for col in cols_100_missing:
    print(f"   • {col}")

df.drop(columns=cols_100_missing, inplace=True)
print(f"\n   Nouvelle forme : {df.shape}")

🗑️ 18 colonnes avec 100% de NaN supprimées :
   • type_handicap
   • economie_s1
   • economie_s2
   • economie_annuel
   • comptabilite_s1
   • comptabilite_s2
   • comptabilite_annuel
   • gestion_s1
   • gestion_s2
   • gestion_annuel
   • note_examen_regional
   • note_examen_national
   • note_controle_continu
   • note_finale_bac
   • mention_bac
   • probabilite_reussite
   • date_mise_a_jour
   • remarques

   Nouvelle forme : (10000, 268)


## 4. Imputation des valeurs manquantes

**Stratégie :**
- **Numériques** : Médiane si |skewness| > 1, sinon Moyenne
- **Catégorielles** : Valeur par défaut sémantique ou Mode

In [5]:
# Résumé des colonnes avec NaN
missing_cols = df.isnull().sum()
missing_cols = missing_cols[missing_cols > 0].sort_values(ascending=False)

print(f"📋 {len(missing_cols)} colonnes avec des valeurs manquantes :\n")
print(f"{'Colonne':<30} {'NaN':>6} {'%':>8} {'Type':<12} {'Stratégie'}")
print('─' * 80)

for col in missing_cols.index:
    n_miss = missing_cols[col]
    pct = n_miss / len(df) * 100
    dtype = str(df[col].dtype)
    if df[col].dtype in ['float64', 'int64']:
        skew = df[col].skew()
        strategy = f"Médiane (skew={skew:.2f})" if abs(skew) > 1 else f"Moyenne (skew={skew:.2f})"
    else:
        strategy = "Mode / Valeur par défaut"
    flag = " ⚠️" if pct > 80 else ""
    print(f"{col:<30} {n_miss:>6} {pct:>7.1f}% {dtype:<12} {strategy}{flag}")

📋 9 colonnes avec des valeurs manquantes :

Colonne                           NaN        % Type         Stratégie
────────────────────────────────────────────────────────────────────────────────
pays_cible                       8344    83.4% object       Mode / Valeur par défaut ⚠️
etablissement_precedent          5925    59.2% object       Mode / Valeur par défaut
matieres_soutien                 4813    48.1% object       Mode / Valeur par défaut
annees_redoublees                4537    45.4% object       Mode / Valeur par défaut
niveau_allemand                  2953    29.5% object       Mode / Valeur par défaut
type_art                         2263    22.6% object       Mode / Valeur par défaut
type_travail                     2239    22.4% object       Mode / Valeur par défaut
type_maladie                     2204    22.0% object       Mode / Valeur par défaut
type_sport                       1421    14.2% object       Mode / Valeur par défaut


In [6]:
# --- Imputation NUMÉRIQUES ---
numeric_missing = [col for col in missing_cols.index if df[col].dtype in ['float64', 'int64']]

imputation_log = []
for col in numeric_missing:
    skewness = df[col].skew()
    if abs(skewness) > 1:
        fill_value = df[col].median()
        method = 'Médiane'
    else:
        fill_value = df[col].mean()
        method = 'Moyenne'
    
    n_filled = df[col].isnull().sum()
    df[col] = df[col].fillna(fill_value)
    imputation_log.append({
        'Colonne': col, 'Méthode': method,
        'Valeur': round(fill_value, 2), 'Skewness': round(skewness, 2),
        'NaN remplis': n_filled
    })

if imputation_log:
    print(f"✅ {len(numeric_missing)} colonnes numériques imputées :\n")
    print(pd.DataFrame(imputation_log).to_string(index=False))
else:
    print("Aucune colonne numérique à imputer.")

Aucune colonne numérique à imputer.


In [7]:
# --- Imputation CATÉGORIELLES ---
cat_missing = [col for col in missing_cols.index if df[col].dtype == 'object']

default_values = {
    'type_sport': 'Aucun', 'type_art': 'Aucun',
    'matieres_soutien': 'Aucun', 'type_maladie': 'Aucune',
    'type_travail': 'Aucun', 'pays_cible': 'Non spécifié',
    'niveau_allemand': 'Non étudié'
}

cat_log = []
for col in cat_missing:
    n_filled = df[col].isnull().sum()
    if col in default_values:
        fill_value = default_values[col]
        method = 'Valeur par défaut'
    else:
        fill_value = df[col].mode()[0] if not df[col].mode().empty else 'Inconnu'
        method = 'Mode'
    df[col] = df[col].fillna(fill_value)
    cat_log.append({'Colonne': col, 'Méthode': method, 'Valeur': fill_value, 'NaN remplis': n_filled})

if cat_log:
    print(f"✅ {len(cat_missing)} colonnes catégorielles imputées :\n")
    print(pd.DataFrame(cat_log).to_string(index=False))
else:
    print("Aucune colonne catégorielle à imputer.")

✅ 9 colonnes catégorielles imputées :

                Colonne           Méthode          Valeur  NaN remplis
             pays_cible Valeur par défaut    Non spécifié         8344
etablissement_precedent              Mode Lycee Ibn Rochd         5925
       matieres_soutien Valeur par défaut           Aucun         4813
      annees_redoublees              Mode       2022-2023         4537
        niveau_allemand Valeur par défaut      Non étudié         2953
               type_art Valeur par défaut           Aucun         2263
           type_travail Valeur par défaut           Aucun         2239
           type_maladie Valeur par défaut          Aucune         2204
             type_sport Valeur par défaut           Aucun         1421


In [8]:
# Vérification
remaining_na = df.isnull().sum().sum()
print(f"🔍 Valeurs manquantes restantes : {remaining_na}")
if remaining_na == 0:
    print("   ✅ Imputation complète — aucune valeur manquante !")
else:
    remaining = df.isnull().sum()
    print(remaining[remaining > 0])

🔍 Valeurs manquantes restantes : 0
   ✅ Imputation complète — aucune valeur manquante !


## 5. Conversion des dates

In [9]:
date_cols = ['date_naissance', 'date_collecte']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d', errors='coerce')
        n_nat = df[col].isna().sum()
        status = f'⚠️ {n_nat} NaT' if n_nat > 0 else '✅ OK'
        print(f"   {col} : {status}")

   date_naissance : ✅ OK
   date_collecte : ✅ OK


## 6. Normalisation des colonnes textuelles

**⚠️ Important** : Les colonnes de dates et d'identifiants sont exclues de la normalisation `title case`  
pour éviter de casser les formats (ex: `STU00001` → `Stu00001`, dates → illisibles).

In [10]:
# Colonnes à exclure du title case (dates, IDs, codes)
exclude_from_title = ['date_naissance', 'date_collecte', 'id_etudiant', 'code_massar', 'id_collecteur']

text_cols = df.select_dtypes(include='object').columns
normalized_count = 0
stripped_count = 0

for col in text_cols:
    if col in exclude_from_title:
        df[col] = df[col].astype(str).str.strip()
        stripped_count += 1
    else:
        df[col] = df[col].astype(str).str.strip().str.title()
        normalized_count += 1

print(f"✅ {normalized_count} colonnes textuelles normalisées (strip + title case)")
print(f"✅ {stripped_count} colonnes préservées (strip uniquement) : {[c for c in exclude_from_title if c in text_cols]}")

✅ 167 colonnes textuelles normalisées (strip + title case)
✅ 3 colonnes préservées (strip uniquement) : ['id_etudiant', 'code_massar', 'id_collecteur']


## 7. Validation des colonnes numériques

- **Notes** → [0, 20] — avec détection automatique de l'échelle (0-100 → converti en 0-20)
- **Taux** → [0, 100]
- **Âge** → [15, 25]

In [11]:
# Notes — avec détection d'échelle
grade_cols = [c for c in df.columns if any(x in c for x in ['_s1', '_s2', '_annuel', 'note_', 'moyenne'])]
for col in grade_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

clipped_grades = 0
scaled_grades = 0
for col in grade_cols:
    if col in df.columns:
        values_above_20 = (df[col] > 20).sum()
        total_non_nan = df[col].notna().sum()
        if total_non_nan > 0 and values_above_20 / total_non_nan > 0.10:
            # Échelle 0-100 détectée → convertir en 0-20
            scaled_grades += values_above_20
            df[col] = df[col] / 5.0
            df[col] = df[col].clip(0, 20)
            print(f"   🔄 {col} : {values_above_20} valeurs converties (échelle 0-100 → 0-20)")
        else:
            out = ((df[col] < 0) | (df[col] > 20)).sum()
            if out > 0:
                clipped_grades += out
                df[col] = df[col].clip(0, 20)

print(f"📝 {len(grade_cols)} colonnes de notes → {scaled_grades} converties, {clipped_grades} clippées dans [0,20]")

# Taux
taux_cols = [c for c in df.columns if 'taux' in c]
clipped_taux = 0
for col in taux_cols:
    if col in df.columns:
        out = ((df[col] < 0) | (df[col] > 100)).sum()
        if out > 0:
            clipped_taux += out
            df[col] = df[col].clip(0, 100)
print(f"📊 {len(taux_cols)} colonnes de taux → {clipped_taux} valeurs clippées dans [0,100]")

# Âge
if 'age' in df.columns:
    age_out = ((df['age'] < 15) | (df['age'] > 25)).sum()
    df['age'] = df['age'].clip(15, 25)
    print(f"🎂 Âge → {age_out} valeurs clippées dans [15,25]")

   🔄 rang_s1 : 5329 valeurs converties (échelle 0-100 → 0-20)
   🔄 rang_s2 : 5352 valeurs converties (échelle 0-100 → 0-20)
   🔄 rang_annuel : 5329 valeurs converties (échelle 0-100 → 0-20)
📝 40 colonnes de notes → 16010 converties, 0 clippées dans [0,20]
📊 4 colonnes de taux → 0 valeurs clippées dans [0,100]
🎂 Âge → 0 valeurs clippées dans [15,25]


## 8. Détection et traitement des outliers (IQR)

**⚠️ Note** : `nombre_freres_soeurs` est traité avec un seuil adapté au contexte marocain (max=10)  
au lieu du clipping IQR automatique (qui donnerait max=4, trop restrictif).

In [12]:
MAX_FRERES_SOEURS = 10
EXCLUDE_FROM_IQR = ['nombre_freres_soeurs']

outlier_cols = [c for c in df.columns if any(x in c for x in ['revenu', 'experience', 'nombre_freres'])]
outlier_cols = [c for c in outlier_cols if df[c].dtype in ['float64', 'int64']]

outlier_report = []
for col in outlier_cols:
    if col in EXCLUDE_FROM_IQR:
        n_out = (df[col] > MAX_FRERES_SOEURS).sum()
        if n_out > 0:
            df[col] = df[col].clip(0, MAX_FRERES_SOEURS)
            outlier_report.append({
                'Colonne': col, 'Bornes': f'[0, {MAX_FRERES_SOEURS}]',
                'Outliers clippés': n_out, 'Méthode': 'Seuil domaine'
            })
    else:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        n_out = ((df[col] < lower) | (df[col] > upper)).sum()
        if n_out > 0:
            df[col] = df[col].clip(lower, upper)
            outlier_report.append({
                'Colonne': col, 'Bornes': f'[{lower:.0f}, {upper:.0f}]',
                'Outliers clippés': n_out, 'Méthode': 'IQR'
            })

if outlier_report:
    print(f"🔎 Outliers clippés :\n")
    print(pd.DataFrame(outlier_report).to_string(index=False))
else:
    print("Aucun outlier détecté.")

🔎 Outliers clippés :

            Colonne         Bornes  Outliers clippés Méthode
revenu_mensuel_pere [-6241, 22705]               241     IQR
revenu_mensuel_mere [-5736, 16609]               205     IQR
    revenu_familial [-6020, 32880]               214     IQR


## 9. Résumé et export

In [13]:
shape_apres = df.shape
missing_apres = df.isnull().sum().sum()

print("=" * 55)
print("📊 RÉSUMÉ DES TRANSFORMATIONS")
print("=" * 55)
print(f"{'':<25} {'AVANT':>12} {'APRÈS':>12}")
print(f"{'─' * 50}")
print(f"{'Lignes':<25} {shape_avant[0]:>12} {shape_apres[0]:>12}")
print(f"{'Colonnes':<25} {shape_avant[1]:>12} {shape_apres[1]:>12}")
print(f"{'Colonnes supprimées':<25} {'':>12} {shape_avant[1] - shape_apres[1]:>12}")
print(f"{'Valeurs manquantes':<25} {missing_avant:>12} {missing_apres:>12}")
print(f"{'─' * 50}")

📊 RÉSUMÉ DES TRANSFORMATIONS
                                 AVANT        APRÈS
──────────────────────────────────────────────────
Lignes                           10000        10000
Colonnes                           286          268
Colonnes supprimées                              18
Valeurs manquantes              204699            0
──────────────────────────────────────────────────


In [14]:
# Export
output_file = 'Morocco_Student_Data_Cleaned.csv'
df.to_csv(output_file, index=False)

print(f"💾 Dataset nettoyé sauvegardé : {output_file}")
print(f"   Taille : {df.shape[0]:,} × {df.shape[1]} colonnes")
print(f"   NaN restants : {df.isnull().sum().sum()}")
print(f"\n➡️  Prochaine étape : exécuter EDA_Advancedا.ipynb")

💾 Dataset nettoyé sauvegardé : Morocco_Student_Data_Cleaned.csv
   Taille : 10,000 × 268 colonnes
   NaN restants : 0

➡️  Prochaine étape : exécuter EDA_Advancedا.ipynb
